<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Snippets_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from __future__ import absolute_import, print_function
from pydrive.auth import *
from pydrive.drive import *
from google.colab import *
from oauth2client.client import *

import glob, os
from os.path import *
from pathlib import Path

drive=None
fileIdDictionary={}
directoryListDict={}
allFilesList=[]
directoryList=[]
fileList=[]

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def mountGoogleDrive():
    '''ToDo...
    fix help
    '''
    from google.colab import drive as gdrive
    gdrive.mount('/content/drive', force_remount=True)
    # contentPath = Path.cwd()
    allFilesList = glob.glob('**', recursive=True)
    print(f'{len(allFilesList)} files in Google Drive')
    os.chdir('/content/drive/My Drive')
    # myDrivePath = Path.cwd()
    # import BashColors
    from BashColors import C
    print(f'cwd: {C.Green}{Path.cwd()}{C.ColorOff}\n')
    
def listGoogleDriveFiles(silent=True):
    '''List all files in the root
    returns fileIdDictionary
    Search query reference:
    https://developers.google.com/drive/v2/web/search-parameters'''
    
    import BashColors
    from BashColors import C
    fileList = drive.ListFile({'q': "title contains '.*' and 'root' in parents"}).GetList() 
    counter=0
    for fil in fileList:
        fileName = fil['title']
        file_id = fil['id']
        fileId = fil['id']
        fileMimeType = fil['mimeType']
        labels = fil['labels'] # Returns a dictionary
        fullPath = abspath(fileName)
        counter+=1
        fileIdDictionary[counter]=(fileName, fil['id'], fullPath)
        if fileMimeType=='application/vnd.google-apps.folder':
            directoryList.append(fullPath)
            directoryListDict[fileName]=(fil['id'], fullPath)


        if not silent:
            if fileMimeType=='application/vnd.google-apps.folder':
                print(f'{C.Blue}{fileName}')
                print(f'{fileMimeType}')
                print(f'{C.White}id: {file_id}')
                for key in labels.keys():
                    print(f'{key}: {labels[key]}')

            else:
                print(f'{C.White}{counter}. {fileName}')
                print(f'{fileMimeType}')
                print(f'id: {file_id}')
                for key in labels.keys():
                    print(f'{key}: {labels[key]}')
            print()
            # if fileMimeType=='application/vnd.google-apps.folder':


mountGoogleDrive()
listGoogleDriveFiles(silent=False)

Mounted at /content/drive
0 files in Google Drive
cwd: /content/drive/My Drive

1. bullshit-2.txt
text/plain
id: 1CqpkZc0nbp9yO_M1X0dK-QFmCGimfuZO
starred: False
hidden: False
trashed: True
restricted: False
viewed: True

2. bullshit-2.txt
text/plain
id: 13MsW4riAWVB1YQ66QmnULr8drfesRyfG
starred: False
hidden: False
trashed: True
restricted: False
viewed: True

testDir
application/vnd.google-apps.folder
id: 1VCJLG6TeQ_FWcHy9QFrlP0XqYEmoC2dJ
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

4. Quickstart.ipynb
application/octet-stream
id: 1HMDPPiGtPU9ZsgRW0flDg0zQInhdKPf9
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

5. GetTPU.py
text/x-python
id: 1ASku3paUKa43wPW1SThlgzMhnMLsTlwb
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

6. control_cartpole_q_learning.py
text/x-python
id: 1PVtMa-7Kg7ng2zM2gKv2YYaotv3Guhs0
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

7. FunctionTimer.

In [ ]:
def selectFile(itemNumber:int):
    '''Returns selected file id number'''
    counter=0
    for item in fileIdDictionary.keys():
        counter+=1
        dict_item = fileIdDictionary[item]
        print(counter, dict_item[0])

    selection = fileIdDictionary[itemNumber]
    selectName = selection[0]
    selectId = selection[1]
    print(f'{selectName}')
    return selectId


idd=selectFile(itemNumber=1)
print(idd)

1 bullshit-2.txt
2 bullshit-2.txt
3 testDir
4 Quickstart.ipynb
5 GetTPU.py
6 control_cartpole_q_learning.py
7 FunctionTimer.py
8 AllUtilities.py
9 GithubRepos.py
10 PT_Fundamentals_Cartpole.ipynb
11 PT
12 BashColors.py
13 TZInfo.py
14 FileUtilities.py
15 CurrentProject
16 CoLab
17 Bash_Colors.ipynb
18 PythonFiles
19 File_Utilities_2.ipynb
20 Colab Notebooks
21 __pycache__
22 Images
23 Swift
bullshit-2.txt
1CqpkZc0nbp9yO_M1X0dK-QFmCGimfuZO


# Saving data to Google Drive

In [ ]:
# import pydrive.drive

def createAndUploadFile(named:str, inDir:str, content:str):
    '''Create & upload a text file.'''
    currentPath=Path.cwd()
    dir_path=join('/content/drive/My Drive', inDir)
    if exists(dir_path):
        os.chdir(dir_path)
        print(Path.cwd)
    else:
        os.mkdir(dir_path)
        os.chdir(dir_path)
        print(Path.cwd())

    uploaded = drive.CreateFile({'title': named})
    uploaded.SetContentString(content)
    # uploaded.Upload()
    listGoogleDriveFiles(silent=False)
    print('Uploaded file with ID {}'.format(uploaded.get('id')))


In [ ]:
file_name='bullshit-2.txt'
fileContent='New content here.'
createAndUploadFile(named=file_name, inDir='testDir', content=fileContent)

<bound method Path.cwd of <class 'pathlib.Path'>>
1. bullshit-2.txt
text/plain
1CqpkZc0nbp9yO_M1X0dK-QFmCGimfuZO
starred: False
hidden: False
trashed: True
restricted: False
viewed: True

2. bullshit-2.txt
text/plain
13MsW4riAWVB1YQ66QmnULr8drfesRyfG
starred: False
hidden: False
trashed: True
restricted: False
viewed: True

3. testDir
application/vnd.google-apps.folder
1VCJLG6TeQ_FWcHy9QFrlP0XqYEmoC2dJ
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

4. Quickstart.ipynb
application/octet-stream
1HMDPPiGtPU9ZsgRW0flDg0zQInhdKPf9
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

5. GetTPU.py
text/x-python
1ASku3paUKa43wPW1SThlgzMhnMLsTlwb
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

6. control_cartpole_q_learning.py
text/x-python
1PVtMa-7Kg7ng2zM2gKv2YYaotv3Guhs0
starred: False
hidden: False
trashed: False
restricted: False
viewed: True

7. FunctionTimer.py
text/x-python
1fEi8DlCC7cmC-kPvM0LJYlUmkqZIeCtn


In [ ]:
def downloadGoogleDriveFile(thisFileId):
    '''Download a file based on its file id'''
    print(thisFileId)
    file_id = thisFileId
    downloaded = drive.CreateFile({'id': file_id})
    print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
downloadFile = selectFile(13)
downloadGoogleDriveFile(downloadFile)

1 bullshit-2.txt
2 bullshit-2.txt
3 testDir
4 Quickstart.ipynb
5 GetTPU.py
6 control_cartpole_q_learning.py
7 FunctionTimer.py
8 AllUtilities.py
9 GithubRepos.py
10 PT_Fundamentals_Cartpole.ipynb
11 PT
12 BashColors.py
13 TZInfo.py
14 FileUtilities.py
15 CurrentProject
16 CoLab
17 Bash_Colors.ipynb
18 PythonFiles
19 File_Utilities_2.ipynb
20 Colab Notebooks
21 __pycache__
22 Images
23 Swift
TZInfo.py
1bPKxKGdMJH0ZNxypirZ5oFzq3Bo5bac5
Downloaded content "# 09/07/2020

import os
import datetime, dateutil
from datetime import *
from dateutil import *
from dateutil.tz import *
from time import ctime
import BashColors
from BashColors import *

class TZI:
    __all__ = sorted(['workingDir', 'tzi','alutianTZone' ,'getCurrentTimeIn', 
                      'getFileStats', 'alaskanTZone', 'hawaiianTZone'
                      'gmtTZone','easternTZone', 'centralTZone', 
                      'mountainTZone', 'pacificTZone'])

    def __init__(self):
        self.tzi = None
        self.workingDi